In [23]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base", 
    downstream_corpus_name="nsmc", 
    downstream_corpus_root_dir="/home/key2317/main/BERT와GPT로 배우는 자연어처리/root/Korpora",
    downstream_model_dir="/home/key2317/main/BERT와GPT로 배우는 자연어처리/nlpbook/checkpoint-doccls",
    learning_rate=5e-5,
    batch_size=32,
)

데이터 내려받기

- 프리트레인을 이미 마친 모델을 다운스트림 데이터로 파인튜닝
- 파인튜닝을 위한 다운스트림 태스크용 데이터를 미리 내려받습니다.
- nsmc : 네이버 영화 리뷰 말뭉치


In [19]:
from Korpora import Korpora

Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:11, 1.31MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 11.0MB/s]                            


Pre-trained 모델 준비

In [20]:
from transformers import BertConfig,BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

토크나이저 준비

- 문장을 토큰 시퀀스로 분석하는 과정을 토큰화라 합니다.
- 그리고 그 토큰화를 수행하는 프로그램을 토크나이저라고 합니다.
- kcbert-base 모델이 사용하는 토크나이저를 준비하는 코드입니다. 

In [21]:
from transformers import BertTokenizer 

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

데이터 로더 준비하기

- Pytorch는 딥러닝 모델 학습을 지원하는 파이썬 라이브러리입니다.
- Pytorch에는 데이터 로더라는게 포함되어 있습니다. 파이토치로 딥러닝 모델을 만들려면 이 데이터로더를 반드시 정의해야 합니다.


- 데이터 로더

  - 데이터를 배치 단위로 모델에 밀어 넣어주는 역할
  - 전체 데이터 중 일부를 뽑아(sample) 배치를 구성합니다.
  - 데이터셋은 데이터 로더의 구성 요소 중 하나입니다. 데이터셋은 여러 인스턴스(문서+레이블)을 포함하고 있습니다. 
  - 데이터 로더가 배치를 만들 때 인스턴스를 뽑는 방식은, 파이토치 사용자가 자유롭게 정할 수 있습니다.

- 배치의 모양은 고정적인 경우가 많습니다. 
- 다시 말해, 동일한 배치에 있는 문장들의 토큰 개수는 서로 같아야 합니다.
- 따라서 배치가 3인 인풋에서 각각 토큰의 길이가 5,3,4라면, 3,4의 길이를 5와 맞추기 위해 0을 삽입하게 됩니다.
- 이 때 이 삽입된 0 token을 padding token이라고 합니다.

문서 분류 데이터 로더 선언

In [22]:
from torch.utils.data import DataLoader, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus,ClassificationDataset 

corpus = NsmcCorpus()

train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset,replacement=False), 
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

NameError: name 'nlpbook' is not defined